## Instrucciones generales 

1. Forme un grupo de **máximo dos estudiantes**
1. Versione su trabajo usando un **repositorio <font color="red">privado</font> de github**. Agregue a sus compañeros y a su profesor (usuario github: phuijse) en la pestaña *Settings/Manage access*. No se aceptarán consultas si la tarea no está en github. No se evaluarán tareas que no estén en github.
1. Se evaluará el **resultado, la profundidad de su análisis y la calidad/orden de sus códigos** en base al último commit antes de la fecha y hora de entrega". Se bonificará a quienes muestren un método de trabajo incremental y ordenado según el histórico de *commits*
1. Sean honestos, ríganse por el [código de ética de la ACM](https://www.acm.org/about-acm/code-of-ethics-in-spanish)

# ¿Es posible explicar la cantidad de billonarios en base al desarrollo país?  <a class="tocSkip"></a>







En 2006 *Daniel Treisman* publicó un artículo titulado [*Russia Billionaries*](https://pubs.aeaweb.org/doi/pdfplus/10.1257/aer.p20161068) en el cual conectó la cantidad de billonarios de un país con ciertos atributos económicos de los mismos. 

Su conclusión principal fue que Rusia tiene una cantidad de billonarios mayor que la que predicen los indicadores económicos

En esta tarea ustedes analizarán datos macroeconómicos para comprobar o refutar los hallazgos de *D. Treisman*

## Datos

Para esta tarea se les provee de un conjunto de datos `billonarios.csv` indexado por país con los siguientes atributos

- `nbillonarios`: La cantidad de billonarios del pais
- `logpibpc`: El logaritmo del Producto Interno Bruto (PIB) per capita del pais
- `logpob`: El logaritmo de la población del pais
- `gatt`: La cantidad de años que el pais está adherido al *General Agreement on Tariffs and Trade* (GATT)

## Modelo 

El objetivo principal de esta tarea es entrenar un modelo de regresión que prediga la cantidad de billonarios en función de los demás atributos

> El número de billonarios es una variable entera y no-negativa. 

Un modelo apropiado en este caso es la [regresión de Poisson](https://en.wikipedia.org/wiki/Poisson_distribution), donde definimos la probabilidad condicional para un pais $i$ como  

$$
p(y_i | x_i ) = \frac{\lambda_i^{y_i}}{y_i!} \exp \left ({-\lambda_i} \right)
$$

con intensidad

$$
\lambda_i = \exp \left (\theta_0 + \sum_{j=1}^M \theta_j x_{ij} \right)
$$

donde 

- $\theta$ es el vector de parámetros que deseamos ajustar 
- $y_i$ y $x_i$ son la cantidad de billonarios y el vector de atributos del país $i$, respectivamente


## Objetivos

- Implemente la regresion de Poisson usando PyMC
    - Considere un prior normal para los parámetros $\theta$
    - Considere una verosimilitud poisson
- Seleccione dos algoritmo de MCMC para entrenar este modelo. 
    - Muestre y analice las trazas de los parámetros.
    - Muestre y analice el número de muestras efectivo, el estadístico de Gelman-Rubin y la función de autocorrelación
    - En base a las trazas y las métricas calibre los hiperparámetros de su algoritmo para tener la mejro convergencia posible
- Análisis
    - Muestre el posterior de los parámetros obtenidos, su mediana y sus percentiles, ¿Cúales son significativamente distintos de cero?
    - Prediga la cantidad de billonarios usando su modelo y la incertidumbre asociada (posterior predictivo). Muestre graficamente sus resultados
    - ¿Cuáles son los 5 países con mayor error en la predicción? ¿Cuáles países tienen un exceso de billonarios? ¿Cúales paises tienen menos billonarios de lo esperado? ¿Qué puede decir sobre Rusia?

In [1]:
import numpy as np
from scipy.special import factorial
import pandas as pd
import matplotlib.pyplot as plt
from math import e
import pymc3 as pm
import theano.tensor as T

In [2]:
datos = pd.read_csv("billonarios.csv")

In [3]:
datos.head(3)

,pais,nbillonarios,logpibpc,logpob,gatt
0,United States,469,10.786021,19.532846,60
1,Canada,25,10.743365,17.319439,0
2,"Bahamas, The",0,10.072139,12.760934,0


In [10]:
from sklearn.datasets import make_moons

x, y = make_moons(100, shuffle=True, noise=0.2, random_state=123456)
x = (x - np.mean(x, keepdims=True))/np.std(x, keepdims=True)

In [ ]:
Y = datos["nbillonarios"]

In [27]:
with pm.Model() as poisson_reg:
    # Datos
    X_shared = pm.Data("x", x)
    # Priors
    pib = pm.Poisson('pib', mu=10, shape=())
    pob = pm.Poisson('pob', mu=10, shape=())
    gatt = pm.Poisson('gatt', mu=10, shape=())
    # Variable determinista
    mu = pm.Deterministic('mu', '''funcion poisson?''')
    # Verosimilitud
    Y_obs = pm.Poisson(name='Y_obs', mu, observed = Y)

In [ ]:
x_test = np.linspace(-5, 5, num=100)

with poisson_reg:
    prior_checks = pm.sample_prior_predictive(samples=50, var_names=['pib', 'pob', 'gatt'])
    
fig, ax = plt.subplot(figsize=(6,3) tight_layout=True)
ax.set_xlabel('X')
ax.set_ylaber('Y')

for pib, pob, gatt in zip(prior_checks['pib'], prior_checks['pob'], prior_checks['gatt']):
    ax.plot(x_test, '''x_test en funcion poisson?''')